# PPG analysis

In [14]:
import os
import tsdf
from paradigma.config import PPGConfig, IMUConfig, SignalQualityFeatureExtractionConfig, SignalQualityFeatureExtractionAccConfig, SignalQualityClassificationConfig, HeartRateExtractionConfig
from paradigma.preprocessing import scan_and_sync_segments, preprocess_ppg_data
from paradigma.heart_rate.heart_rate_analysis import extract_signal_quality_features, signal_quality_classification, estimate_heart_rate, aggregate_heart_rate

In [2]:
# Cell has the tag 'parameters'
branch = 'heart_rate'
sensor = 'ppg'

path_to_data =  '../../../tests/data'
path_to_classifier = os.path.join(path_to_data, '0.classification', sensor)
path_to_sensor_data = os.path.join(path_to_data, '1.prepared_data')
path_to_preprocessed_data = os.path.join(path_to_data, '2.preprocessed_data', sensor)
path_to_quality_features = os.path.join(path_to_data, '3.extracted_features', branch)
path_to_signal_quality = os.path.join(path_to_data, '4.predictions', branch)
path_to_hr_estimate = os.path.join(path_to_data, '5.quantification', branch)
path_to_hr_aggregation = os.path.join(path_to_data, '6.aggregation', branch)

aggregation_filename = 'heart_rate_aggregates.json'

## Load the data

In [ ]:
config_ppg = PPGConfig()
config_imu = IMUConfig()
# First, we scan and synchronize the data based on the metadatas
metadatas_ppg, metadatas_imu = scan_and_sync_segments(os.path.join(path_to_sensor_data, sensor), os.path.join(path_to_sensor_data, 'imu'))
# Then, we load the data
metadata_time_ppg = metadatas_ppg[0][config_ppg.time_filename]
metadata_values_ppg = metadatas_ppg[0][config_ppg.values_filename]
df_ppg = tsdf.load_dataframe_from_binaries([metadata_time_ppg, metadata_values_ppg], tsdf.constants.ConcatenationType.columns)

metadata_time_imu = metadatas_imu[0][config_imu.time_filename]
metadata_values_imu = metadatas_imu[0][config_imu.values_filename]
df_imu = tsdf.load_dataframe_from_binaries([metadata_time_imu, metadata_values_imu], tsdf.constants.ConcatenationType.columns)

## Preprocessing

In [ ]:
df_ppg_proc, df_acc_proc = preprocess_ppg_data(df_ppg, df_imu, config_ppg, config_imu)

## Extract signal quality features

In [ ]:
config_ppg = SignalQualityFeatureExtractionConfig()
config_acc = SignalQualityFeatureExtractionAccConfig()
df_features = extract_signal_quality_features(config_ppg, df_ppg_proc, config_acc, df_acc_proc)

## Signal quality classification

In [ ]:
config = SignalQualityClassificationConfig()
df_sqa = signal_quality_classification(df_features, config, path_to_classifier)

## Heart rate estimation

In [ ]:
config = HeartRateExtractionConfig()
df_hr = estimate_heart_rate(df_sqa, df_ppg_proc, config)

## Heart rate aggregation

In [ ]:
hr_values = df_hr['heart_rate'].values
df_hr_agg = aggregate_heart_rate(hr_values, aggregates = ['mode', '99p'])